Code by James A White

### Skipped Validation Set
In the interest of keeping this analysis fairly easy to follow the validation set has been excluded in favour of a test set only. It is standard practice to have a train, validation and test set if there are enough available data, as this generally gives a more reliable estimate of how the algorithm will perform in production. 

#### This is by no means an exaustive analysis and is only a rough representation of what can be done for this type of problem. There are certain obvious steps which have not been included some of which are outlined near the end of this notebook. Although this notebook outlines the process for facilitating predictive maintenance, similar logic can be used for other time series regression or even classification problems.

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.signal as ss
import matplotlib.pyplot as plt
import seaborn as sns

import optuna

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost
import catboost

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

plt.rcParams['figure.figsize'] = 20, 20

In [ ]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i+1) for i in range(0,21)]
col_names = index_names + setting_names + sensor_names
directory = r'C:\Users\ecf\Documents\predictive_maintenance'
train_df = pd.read_csv(directory+r'\train_FD003.txt', 
                    sep='\s+', 
                    header=None,
                    index_col=False,
                    names=col_names)
train = train_df.copy()

test_df = pd.read_csv(directory+r'\test_FD003.txt', 
                    sep='\s+', 
                    header=None,
                    index_col=False,
                    names=col_names)
test = test_df.copy()

y_test = pd.read_csv(directory+r'\RUL_FD003.txt', 
                     sep='\s+', 
                     header=None,
                     index_col=False,
                     names=['RUL'])

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.loc[:,'s_1':].describe().transpose()

In [ ]:
def add_remaining_useful_life(df):

    grouped_by_unit = df.groupby(by='unit_number')
    max_cycle = grouped_by_unit['time_cycles'].max()
    
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_number', right_index=True)
    
    # Calculate remaining useful life for each row
    remaining_useful_life = result_frame["max_cycle"] - result_frame['time_cycles']
    result_frame["RUL"] = remaining_useful_life
    
    # drop max_cycle as it's no longer needed
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame

In [ ]:
train = add_remaining_useful_life(train)

In [ ]:
train.head()

In [ ]:
max_ruls = train.groupby('unit_number').max().reset_index()

In [ ]:
max_ruls.head()

In [ ]:
max_ruls['RUL'].hist(bins=20)
plt.xlabel('RUL')
plt.ylabel('frequency')
print(max_ruls['RUL'].max())

### Distribution of RUL
It looks log-normal with the majority of the max RUL data in the 150-250 range. One of a few insights from this is that if there many were more simulations we could be fairly confident that the RUL would never be greater than 550 cycles. This information could be used to clip predictions at a particular maximum which may make the algorithm more reliable/accurate in production.

### Visualise Sensor Signals
Now to look at what the sensor signals look like, this will help in determining "good" and "bad" sensors or sensors that contain a lot of information vs ones that don't.

In [ ]:
def plot_signal(df, signal_name):
    plt.figure(figsize=(13,5))
    for i in df['unit_number'].unique():
        if (i % 10 == 0):  
            plt.plot('RUL', signal_name, 
                     data=df[df['unit_number']==i])
    plt.xlim(250, 0)  # reverse the x-axis so RUL counts down to zero
    plt.xticks(np.arange(0, 300, 25))
    plt.ylabel(signal_name)
    plt.xlabel('Remaining Useful Life')
    plt.show()

def plot_smooth_signal(df, smothed_signal):
    plt.figure(figsize=(13,5))
    for i in df['unit_number'].unique():
        if i == 10:#(i % 10 == 0):  
            plt.plot(df[df['unit_number']==i]['RUL'], smothed_signal)
    plt.xlim(250, 0)  # reverse the x-axis so RUL counts down to zero
    plt.xticks(np.arange(0, 300, 25))
    plt.ylabel(signal_name)
    plt.xlabel('Remaining Useful Life')
    plt.show()

In [ ]:
for sensor in ['s_1',
               's_2',
               's_3',
               's_4',
               's_5',
               's_6',
               's_7',
               's_8',
               's_9',
               's_10',
               's_11',
               's_12',
               's_13',
               's_14',
               's_15',
               's_16',
               's_17',
               's_18',
               's_19',
               's_20',
               's_21']:
    try:
        plot_signal(train, sensor)
    except:
        pass

### Brief Sensor Analysis
From looking at the above it seems as though sensors 1, 5, 16, 18 and 19 have very little to no information to determine ot help predict the RUL. These will be removed before prediction below to help the speed and generalisation of the algorithm.

In [ ]:
# prep data
# drop unwanted columns and split target variable from training set
drop_sensors = ['s_1','s_5','s_16','s_18','s_19']
drop_labels = index_names+setting_names+drop_sensors
remaining_sensors = ['s_2', 's_3', 's_4', 's_6', 's_7', 's_8', 's_9', 's_10',
       's_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21']

X_train = train.drop(drop_labels, axis=1)
y_train = X_train.pop('RUL')

X_test = test.groupby('unit_number').last().reset_index().drop(drop_labels, axis=1)

#### Defining the evalution function for the RMSE metric

In [ ]:
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    print('{} set RMSE:{}, R2:{}'.format(label, rmse, variance))

### Algorithm Training
Below a few algorithms are tested to see which will perform best on the test set, this is just a small list and more algorithms should be tested to achieve the best results. There has been no feature scaling because we are using tree based algorithms only, however, this is usually a necessary step to test other algorithms.

In [ ]:
xgb = xgboost.XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)

# predict and evaluate
y_hat_train = xgb.predict(X_train)
evaluate(y_train, y_hat_train, 'train')

y_hat_test = xgb.predict(X_test)
evaluate(y_test, y_hat_test)

In [ ]:
catb = catboost.CatBoostRegressor(verbose=False, random_state=42)
catb.fit(X_train, y_train)

# predict and evaluate
y_hat_train = catb.predict(X_train)
evaluate(y_train, y_hat_train, 'train')

y_hat_test = catb.predict(X_test)
evaluate(y_test, y_hat_test)

In [ ]:
rf = RandomForestRegressor(max_features="sqrt", random_state=42)
rf.fit(X_train, y_train)

# predict and evaluate
y_hat_train = rf.predict(X_train)
evaluate(y_train, y_hat_train, 'train')

y_hat_test = rf.predict(X_test)
evaluate(y_test, y_hat_test)

### Visualise Results
It is always important to visualise the predictions vs the results as a single metric doesn't always tell the whole story. For example most predictions may be good but there may be one prediction that is a large outlier that would be unacceptable to put into production for particular applications.

In [ ]:
def plot_results(y_test, y_hat_test):
    width = 0.8

    actuals = [int(x) for x in y_test.values]
    predictions = list(y_hat_test)

    indices = np.arange(len(y_hat_test))

    plt.figure(figsize=(60,20))

    plt.bar(indices, actuals, width=width, 
            color='b', label='Actual RUL')
    plt.bar([i for i in indices], predictions, 
            width=0.5*width, color='r', alpha=0.7, label='Predicted RUL')

    plt.legend(prop={'size': 30})
    plt.tick_params(labelsize=30)

    plt.show()

In [ ]:
plot_results(y_test, y_hat_test)

### RUL Clipping

Apply RUL clipping making the max RUL value 115, this helps the predictions as we will see below. It makes sense because the sensor values with a 200 RUL are quite similar to those with a 115 RUL so the algorithm will not be able to distinguish between these well. Also the maximum RUL in the test set is 115 which is about the point where the sensors really start to change (referring to the line graphs above).

In [ ]:
drop_sensors = ['s_1','s_5','s_16','s_18','s_19']
drop_labels = index_names+setting_names+drop_sensors
remaining_sensors = ['s_2', 's_3', 's_4', 's_6', 's_7', 's_8', 's_9', 's_10',
       's_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21']

X_train = train.drop(drop_labels, axis=1)
y_train = X_train.pop('RUL')
y_train_clipped = y_train.clip(upper=115)  # apply RUL clipping

X_test = test.groupby('unit_number').last().reset_index().drop(drop_labels, axis=1)

In [ ]:
rf = RandomForestRegressor(max_features="sqrt", random_state=42)
rf.fit(X_train, y_train_clipped)

# predict and evaluate
y_hat_train = rf.predict(X_train)
evaluate(y_train_clipped, y_hat_train, 'train')

y_hat_test = rf.predict(X_test)
evaluate(y_test, y_hat_test)

### A big improvement, more than halved the RMSE!

In [ ]:
width = 0.8

actuals = [int(x) for x in y_test.values]
predictions = list(y_hat_test)

indices = np.arange(len(y_hat_test))

plt.figure(figsize=(60,20))

plt.bar(indices, actuals, width=width, 
        color='b', label='Actual RUL')
plt.bar([i for i in indices], predictions, 
        width=0.5*width, color='r', alpha=0.7, label='Predicted RUL')

plt.legend(prop={'size': 30})
plt.tick_params(labelsize=30)

plt.show()

### Signal Smoothing
Visualising the signals (line graphs above) we can see that the signals are quite noisy. Another thing we can try to improve the prediction results is to remove some of the noise. To help remove the noise we can implement various filters, I've chosen to use the Savitzky-Golay filter. This basically fits regressions to subsets of the data and has the option to differentiate the data as well. It works quite well for this particular case but may not be the best filter. Again similar to the algorithm selection above many filters should be tested to achieve the best results.

In [ ]:
# signal smoothing function
def apply_scipy_filter(df, scipy_filter):
    for unit in df['unit_number'].unique():
        for sensor in df.loc[:,'s_1':]:
            if sensor != 'RUL':
    
                df.loc[df['unit_number']==unit, sensor] = scipy_filter(df.loc[df['unit_number']==unit, sensor],
                                                                              window_length=19, 
                                                                              polyorder=1,
                                                                              deriv=0,
                                                                              mode='interp') 
    return df

### Filter Application
After some quick testing the best results were achieved by applying the filter 3 times to the data. Again, there may be better combinations or ways of filtering the data but only a few different combinations were explored here. Certain optimisation algorithms can help in the selection process.

In [ ]:
train = apply_scipy_filter(train, ss.savgol_filter)
train = apply_scipy_filter(train, ss.savgol_filter)
train = apply_scipy_filter(train, ss.savgol_filter)

In [ ]:
test = apply_scipy_filter(test, ss.savgol_filter)
test = apply_scipy_filter(test, ss.savgol_filter)
test = apply_scipy_filter(test, ss.savgol_filter)

### Visualise Sensor Signals
We can see from the graphs below that the filters have worked quite well removing a lot of the noise in the signal data.

In [ ]:
for sensor in ['s_1',
               's_2',
               's_3',
               's_4',
               's_5',
               's_6',
               's_7',
               's_8',
               's_9',
               's_10',
               's_11',
               's_12',
               's_13',
               's_14',
               's_15',
               's_16',
               's_17',
               's_18',
               's_19',
               's_20',
               's_21']:
    try:
        plot_signal(train, sensor)
    except:
        pass

In [ ]:
# prep data
# drop unwanted columns and split target variable from training set
drop_sensors = ['s_1','s_5','s_16','s_18','s_19']  # s_6 and s_10 get the benefit of the doubt
drop_labels = index_names+setting_names+drop_sensors
remaining_sensors = ['s_2', 's_3', 's_4', 's_6', 's_7', 's_8', 's_9', 's_10',
       's_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21']

X_train = train.drop(drop_labels, axis=1)
y_train = X_train.pop('RUL')
y_train_clipped = y_train.clip(upper=125)  # apply RUL clipping

# Since the true RUL values for the test set are only provided for the last time cycle of each engine, 
# the test set is subsetted to represent the same
X_test = test.groupby('unit_number').last().reset_index().drop(drop_labels, axis=1)

In [ ]:
rf = RandomForestRegressor(max_features="sqrt", random_state=42)
rf.fit(X_train, y_train_clipped)

# predict and evaluate
y_hat_train = rf.predict(X_train)
evaluate(y_train_clipped, y_hat_train, 'train')

y_hat_test = rf.predict(X_test)
evaluate(y_test, y_hat_test)

### Another Improvement
Here we get another improvement although quite a bit more modest than the RUL clipping it still has lowered the RMSE a significant amount and improved the R2 score.

In [ ]:
plot_results(y_test, y_hat_test)

### Random Forest with Bayesian Optimisation
Algorithm not included - contact 4CDA

In [ ]:
# Params found using Bayesian Optimisation
rf = RandomForestRegressor(n_estimators = 32,
                           max_depth = 22,
                           min_samples_split = 6,
                           max_features = 1,
                           min_samples_leaf = 8,
                           random_state = 42)
rf.fit(X_train, y_train_clipped)

# predict and evaluate
y_hat_train = rf.predict(X_train)
evaluate(y_train_clipped, y_hat_train, 'train')

y_hat_test = rf.predict(X_test)
evaluate(y_test, y_hat_test)

In [ ]:
plot_results(y_test, y_hat_test)

### BONUS: XGBoost with optimization
Algorithm not included - contact 4CDA

In [ ]:
# Params found using Bayesian Optimisation
xgb = xgboost.XGBRegressor(n_estimators=32, 
                           max_depth=22,
                           learning_rate=0.287874962131598,
                           reg_lambda=5,
                           gamma=0.4510565094063483,
                           random_state=42)
xgb.fit(X_train, y_train_clipped)

# predict and evaluate
y_hat_train = xgb.predict(X_train)
evaluate(y_train_clipped, y_hat_train, 'train')

y_hat_test = xgb.predict(X_test)
evaluate(y_test, y_hat_test)

In [ ]:
plot(y_test, y_hat_test)